In [33]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import pyproj
import numpy as np

In [19]:
residences = pd.read_csv('../data/Schema/residences_table.csv', usecols=[1,2,3,4])
contact_points = pd.read_csv('../data/Schema/contact_points_table.csv', usecols=[1,2,3,4,5,6,7,8,9])
survivors = pd.read_csv('../data/Schema/survivors_table.csv', usecols=[1])
contact_info = pd.read_csv('../data/Schema/contact_info_table.csv', usecols=[1,2])

In [20]:
print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

CONTACT POINTS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   residence_id  1886 non-null   object 
 1   survivor_id   1032 non-null   float64
 2   date          1896 non-null   object 
 3   water_lvl     272 non-null    object 
 4   damage_lvl    1108 non-null   object 
 5   notes1        1553 non-null   object 
 6   notes2        1277 non-null   object 
 7   notes3        301 non-null    object 
 8   intaker       1896 non-null   object 
dtypes: float64(1), object(8)
memory usage: 133.4+ KB

RESIDENCES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    880 non-null    object 
 1   zipcode    880 non-null    int64  
 2   latitude   880 non-null    float64
 3   longitude  880 non-null    floa

(None, None, None, None, None, None, None, None)

In [21]:
contact_info['contact_info'] = contact_info['contact_info'].str.replace('-','', regex=False)
contact_info = contact_info.drop_duplicates()
contact_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1218 entries, 0 to 1734
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   survivor_id   1218 non-null   int64 
 1   contact_info  1218 non-null   object
dtypes: int64(1), object(1)
memory usage: 28.5+ KB


In [22]:
residences_geo = gpd.GeoDataFrame(residences, geometry=gpd.points_from_xy(residences.longitude, residences.latitude), crs='EPSG:4326')

In [23]:
council_districts = gpd.read_file('../data/Schema/Council District Outlines (GIS).geojson')

In [24]:
residences_geo = gpd.sjoin(residences_geo, council_districts, how='left', op='within')
residences_geo.to_csv('../data/Schema/residences_geo.csv')

In [39]:
contact_points['water_lvl'].value_counts()

2       79
3       45
1       39
0.5     17
3.5     15
2.5     13
6       13
0.6     12
1.5      9
7        5
4        4
5        4
0.75     2
8        2
9        2
0.7      2
1.3      2
6.5      2
0.9      1
1.4      1
4.5      1
Name: water_lvl, dtype: int64

In [26]:
contact_points.loc[contact_points['damage_lvl']=='major', 'damage_lvl'] = 'Major'

In [30]:
contact_points.loc[contact_points['damage_lvl']=='minor', 'damage_lvl'] = 'Minor'
contact_points.loc[contact_points['damage_lvl']=='affected', 'damage_lvl'] = 'Affected'
contact_points.loc[contact_points['damage_lvl']=='DESTROYED', 'damage_lvl'] = 'Destroyed'
contact_points.loc[contact_points['damage_lvl']=='mon', 'damage_lvl'] = 'Minor'

In [38]:
contact_points.loc[contact_points['water_lvl']=='water_living', 'water_lvl'] = np.nan
contact_points.loc[contact_points['water_lvl']=='water_below,trees,other', 'water_lvl'] = np.nan

In [40]:
contact_points.to_csv('../data/contact_points_table.csv')
contact_info.to_csv('../data/contact_info_table.csv')